# Recommender Systems

In [1]:
from random import gauss as gs, uniform as uni, seed
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

![Netflix](https://miro.medium.com/max/1400/1*jlQxemlP9Yim_rWTqlCFDQ.png)

![Amazon](images/amazon_recommender.png)

## Agenda

- describe the difference between content-based and collaborative-filtering algorithms
- explain and use the cosine similarity metric
- describe the algorithm of alternating least-squares

## Intro

Recommender systems can be classified along various lines. One fundamental distinction is **content-based** vs. **collaborative-filtering** systems.

To illustrate this, consider two different strategies: (a) I recommend items to you that are *similar to other items* you've used/bought/read/watched; and (b) I recommend items to you that people *similar to you* have used/bought/read/watched. The first is the **content-based strategy**; the second is the **collaborative-filtering strategy**. 

Another distinction drawn is in whether (a) the system uses existing ratings to compute user-user or item-item similarity, or (b) the system uses machine learning techniques to make predictions. Recommenders of the first sort are called **memory-based**; recommenders of the second sort are called **model-based**.

## Content-Based Systems

The basic idea here is to recommend items to a user that are *similar to* items that the user has already enjoyed. Suppose we represent TV shows as rows, where the columns represent various features of these TV shows. These features might be things like the presence of a certain actor or the show fitting into a particular genre etc. We'll just use binary features here, perhaps the result of some one-hot encoding:

In [2]:
tv_shows = np.array([[0, 1, 1, 0, 1, 1, 1],
                   [0, 0, 0, 1, 1, 1, 0],
                   [1, 1, 1, 0, 0, 1, 1],
                   [0, 1, 1, 1, 0, 0, 1]])

tv_shows

array([[0, 1, 1, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 0],
       [1, 1, 1, 0, 0, 1, 1],
       [0, 1, 1, 1, 0, 0, 1]])

Bob likes the TV Show represented by Row \#1. Which show (row) should we recommend to Bob?

One natural way of measuring the similarity between two vectors is by the **cosine of the angle between them**. Two points near one another in feature space will correspond to vectors that nearly overlap, i.e. vectors that describe a small angle $\theta$. And as $\theta$ decreases, $\cos(\theta)$ *increases*. So we'll be looking for large values of the cosine (which ranges between -1 and 1). We can also think of the cosine between two vectors as the *projection of one vector onto the other*:

![image.png](https://www.oreilly.com/api/v2/epubs/9781788295758/files/assets/2b4a7a82-ad4c-4b2a-b808-e423a334de6f.png)

We can use this metric easily if we treat our rows (the items we're comparing for similarity) as vectors: We can calculate the cosine of the angle $\theta$ between two vectors $\vec{a}$ and $\vec{b}$ as follows: $\cos(\theta) = \frac{\vec{a}\cdot\vec{b}}{|\vec{a}||\vec{b}|}$

In [3]:
numerators = np.array([tv_shows[0].dot(tv_show) for tv_show in tv_shows[1:]])
denominators = np.array([np.sqrt(sum(tv_shows[0]**2)) *\
                         np.sqrt(sum(tv_show**2)) for tv_show in tv_shows[1:]])

numerators / denominators

array([0.51639778, 0.8       , 0.67082039])

Since the cosine similarity to Row \#1 is highest for Row \#3, we would recommend this TV show.

***

## Collaborative Filtering

Now the idea is to recommend items to a user based on what *similar* users have enjoyed. Suppose we have the following recording of explicit ratings of five items by three users:

In [5]:
users = np.array([[5, 4, 3, 4, 5], [3, 1, 1, 2, 5], [4, 2, 3, 1, 4]])

new_user = np.array([5, 0, 0, 0, 0])
users

array([[5, 4, 3, 4, 5],
       [3, 1, 1, 2, 5],
       [4, 2, 3, 1, 4]])

In [6]:
new_user

array([5, 0, 0, 0, 0])

To which user is `new_user` most similar?

One metric is cosine similarity:

In [7]:
new_user_mag = 5

numerators = np.array([new_user.dot(user) for user in users])
denominators = np.array([new_user_mag * np.sqrt(sum(user**2))\
                         for user in users])

numerators / denominators

array([0.52414242, 0.47434165, 0.58976782])

But we could also use another metric, such as Pearson Correlation:

In [8]:
[np.corrcoef(new_user, user)[0, 1] for user in users]

[0.5345224838248488, 0.20044593143431824, 0.5144957554275266]

For more on content-based vs. collaborative systems, see [this Wikipedia article](https://en.wikipedia.org/wiki/Collaborative_filtering) and [this blog post](https://towardsdatascience.com/recommendation-systems-models-and-evaluation-84944a84fb8e). [This post](https://dataconomy.com/2015/03/an-introduction-to-recommendation-engines/) on dataconomy is also useful.

***

## Matrix Factorization

Suppose we start with a matrix $R$ of users and products, where each cell records the ranking the relevant user gave to the relevant product. Very often we'll be able to record this data as a sparse matrix, because many users will not have ranked many items.

Imagine factoring this matrix into a user matrix $P$ and an item matrix $Q^T$: $R = PQ^T$. What would the shapes of $P$ and $Q^T$ be? Clearly $P$ must have as many rows as $R$, which is just the number of users who have given ratings. Similarly, $Q^T$ must have as many columns as $R$, which is just the number of items that have received ratings. We also know that the number of columns of $P$ must match the number of rows of $Q^T$ for the factorization to be possible, but this number could really be anything. In practice this will be a small number, and for reasons that will emerge shortly let's refer to these dimensions as **latent features** of the items in $R$. If $p$ is a row of $P$, i.e. a user-vector, and $q$ is a column of $Q^T$, i.e. an item-vector, then $p$ will record the user's particular weights or *preferences* with respect to the latent features, while $q$ will record how the item ranks with respect to those same latent features. This in turn means that we could predict a user's ranking of a particular item simply by calculating the dot-product of $p$ and $q$! 

If we could effect such a factorization, $R = PQ^T$, then we could calculate *all* predictions, i.e. fill in the gaps in $R$, by solving for $P$ and $Q$.

The isolation of these latent features can be achieved in various ways. But this is at heart a matter of **dimensionality reduction**, and so one way is with the [SVD](https://hackernoon.com/introduction-to-recommender-system-part-1-collaborative-filtering-singular-value-decomposition-44c9659c5e75).

An alternative is to use the method of Alternating Least Squares.

### Alternating Least-Squares (ALS)

ALS recommendation systems are often implemented in Spark architectures because of the appropriateness for distributed computing. ALS systems often involve very large datasets (consider how much data the recommendation engine for NETFLIX must have, for example!), and it is often useful to store them as sparse matrices, which Spark's ML library can handle. In fact, Spark's mllib even includes a "Rating" datatype! ALS is **collaborative** and **model-based**, and is especially useful for working with *implicit* ratings.

We're looking for two matrices (a user matrix and an item matrix) into which we can factor our ratings matrix. We can't of course solve for two matrices at once. But here's what we can do:

Make guesses of the values for $P$ and $Q$. Then hold the values of one *constant* so that we can optimize for the values of the other!

Basically this converts our problem into a familiar *least-squares* problem. See [this page](https://textbooks.math.gatech.edu/ila/least-squares.html) and [this page](https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/) for more details, but here's the basic idea:

If we have an equation $Ax = b$ for *non-square* $A$, then we have:

$A^TAx = A^Tb$ <br/>
Thus: <br/>
$x = (A^TA)^{-1}A^Tb$

This $(A^TA)^{-1}A^T$ **is the pseudo-inverse of** $A$. We encountered this before in our whirlwind tour of linear algebra.

In [9]:
np.random.seed(42)

A = np.random.rand(5, 5)
b = np.random.rand(5, 1)

In [10]:
np.linalg.inv(A.T.dot(A)).dot(A.T).dot(b)

array([[-226.17808974],
       [ 218.48756728],
       [-362.56506519],
       [ 147.90541489],
       [ 350.1459068 ]])

The `numpy` library has a shortcut for this: `numpy.linalg.pinv()`:

In [11]:
np.linalg.pinv(A).dot(b)

array([[-226.17808981],
       [ 218.48756735],
       [-362.56506531],
       [ 147.90541493],
       [ 350.14590691]])

"When we talk about collaborative filtering for recommender systems we want to solve the problem of our original matrix having millions of different dimensions, but our 'tastes' not being nearly as complex. Even if i’ve \[sic\] viewed hundreds of items they might just express a couple of different tastes. Here we can actually use matrix factorization to mathematically reduce the dimensionality of our original 'all users by all items' matrix into something much smaller that represents 'all items by some taste dimensions' and 'all users by some taste dimensions'. These dimensions are called ***latent or hidden features*** and we learn them from our data" ([Medium article: "ALS Implicit Collaborative Filtering"](https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe)).

#### Simple Example

Suppose Max and Erin have rated five films:

In [12]:
ratings_arr = pd.DataFrame([[0, 1, 0, 0, 4], [0, 0, 0, 5, 0]],\
                           index=['max', 'erin'],
             columns=['film' + str(i) for i in range(1, 6)])
ratings_arr

,film1,film2,film3,film4,film5
max,0,1,0,0,4
erin,0,0,0,5,0


Suppose now that we isolate ten latent features of these films, and that we can capture our users, i.e. the tastes of Max and Erin, according to these features. (We'll just fill out a matrix randomly.)

In [13]:
seed(100)
users = []

for _ in range(2):
    user = []
    for _ in range(10):
        user.append(gs(0, 1))
    users.append(user)
users_arr = np.array(users)

In [14]:
users_arr

array([[ 0.67155333,  0.87331967,  0.20361655, -1.55034921, -0.12059128,
        -1.05927574,  0.38143697, -1.17342904,  0.96637182,  0.53248343],
       [ 2.22041991,  0.68901284,  0.85436449, -0.29504752, -0.62335055,
         1.5917369 ,  0.17920475,  0.60086339,  0.3474319 ,  0.85537186]])

In [15]:
pd.DataFrame(users_arr)

,0,1,2,3,4,5,6,7,8,9
0,0.671553,0.873320,0.203617,-1.550349,-0.120591,-1.059276,0.381437,-1.173429,0.966372,0.532483
1,2.220420,0.689013,0.854364,-0.295048,-0.623351,1.591737,0.179205,0.600863,0.347432,0.855372


Now we'll make another random matrix that expresses *our items* in terms of these latent features.

In [16]:
seed(100)
items = []

for _ in range(5):
    item = []
    for _ in range(10):
        item.append(gs(0, 1))
    items.append(item)
items_arr = np.array(items)

In [17]:
items_arr

array([[ 0.67155333,  0.87331967,  0.20361655, -1.55034921, -0.12059128,
        -1.05927574,  0.38143697, -1.17342904,  0.96637182,  0.53248343],
       [ 2.22041991,  0.68901284,  0.85436449, -0.29504752, -0.62335055,
         1.5917369 ,  0.17920475,  0.60086339,  0.3474319 ,  0.85537186],
       [-1.80291827, -1.83311295,  0.60911087,  2.4250145 , -2.02233576,
        -0.73382756,  0.24510831, -0.53817586, -0.30637608, -0.41367264],
       [ 1.0027436 ,  0.03558136,  0.19013362, -1.27827915,  0.67648704,
         1.79506722,  0.63054322, -0.37947302, -1.35033057,  0.45576721],
       [ 0.42542416, -0.29962041, -2.48035968, -0.87457154, -1.23050164,
        -1.00629648,  0.1857537 , -1.174924  , -0.33108494,  1.29437514]])

In [19]:
pd.DataFrame(items_arr).T

,0,1,2,3,4
0,0.671553,2.220420,-1.802918,1.002744,0.425424
1,0.873320,0.689013,-1.833113,0.035581,-0.299620
2,0.203617,0.854364,0.609111,0.190134,-2.480360
3,-1.550349,-0.295048,2.425014,-1.278279,-0.874572
4,-0.120591,-0.623351,-2.022336,0.676487,-1.230502
5,-1.059276,1.591737,-0.733828,1.795067,-1.006296
6,0.381437,0.179205,0.245108,0.630543,0.185754
7,-1.173429,0.600863,-0.538176,-0.379473,-1.174924
8,0.966372,0.347432,-0.306376,-1.350331,-0.331085
9,0.532483,0.855372,-0.413673,0.455767,1.294375


In [20]:
users_arr.shape

(2, 10)

In [21]:
items_arr.shape

(5, 10)

To construct our large users-by-items matrix, we'll simply take the product of our two random matrices.

In [22]:
users_arr.dot(items_arr.T)

array([[ 7.53516384,  1.26783511, -5.21738432,  0.36547776,  3.90803828],
       [ 1.26783511, 10.38970834, -6.10853643,  5.03189793, -1.63817674]])

In [23]:
pd.DataFrame(users_arr.dot(items_arr.T))

,0,1,2,3,4
0,7.535164,1.267835,-5.217384,0.365478,3.908038
1,1.267835,10.389708,-6.108536,5.031898,-1.638177


In [24]:
ratings_arr

,film1,film2,film3,film4,film5
max,0,1,0,0,4
erin,0,0,0,5,0


Now here's where the ALS really kicks in: We'll solve for Max's and Erin's preference vectors by multiplying the pseudo-inverse of the items array by their respective ratings vectors:

In [27]:
max_pref = np.linalg.pinv(items_arr).dot(ratings_arr.loc['max', :])
print(max_pref)
erin_pref = np.linalg.pinv(items_arr).dot(ratings_arr.loc['erin', :])
print(erin_pref)

[ 0.47099573 -0.11214938 -0.98527859  0.09266666 -0.64747458  0.00854642
 -0.11601818  0.09586291 -0.08579953  0.55693307]
[-0.23238655 -0.55138144  0.5415508  -0.71191248  0.27767197  0.81157767
  0.78557579 -1.03623954 -1.25490079  0.02606555]


In [28]:
items_arr.T.shape

(10, 5)

In [29]:
max_pref

array([ 0.47099573, -0.11214938, -0.98527859,  0.09266666, -0.64747458,
        0.00854642, -0.11601818,  0.09586291, -0.08579953,  0.55693307])

In [30]:
erin_pref

array([-0.23238655, -0.55138144,  0.5415508 , -0.71191248,  0.27767197,
        0.81157767,  0.78557579, -1.03623954, -1.25490079,  0.02606555])

We'll predict (or, in this case, reproduce) Max's and Erin's ratings for films by simply multiplying their preference vectors by the transpose of the items array:

In [31]:
newmax = max_pref.dot(items_arr.T)
newmax

array([-7.91033905e-16,  1.00000000e+00,  0.00000000e+00,  2.02615702e-15,
        4.00000000e+00])

This lines up with the ratings with which we began.

In [32]:
newerin = erin_pref.dot(items_arr.T)
newerin

array([ 4.44089210e-16,  2.44249065e-15,  3.33066907e-16,  5.00000000e+00,
       -1.55431223e-15])

In [33]:
ratings_arr

,film1,film2,film3,film4,film5
max,0,1,0,0,4
erin,0,0,0,5,0


Ditto!

We'll make a quick error calculation:

In [34]:
guess = np.vstack([newmax, newerin])

err = 0
for i in range(2):
    for j in range(len(ratings_arr.values[i, :])):
        if ratings_arr.values[i, j] != 0:
            err += (ratings_arr.values[i, j] - guess[i, j])**2
print(err)

3.318146182585881e-29


#### Second Example

In [ ]:
# Users: m x n (m users)
# Items: r x n (r items)
# Ratings: m x r

In [35]:
# If P = users and Q = items, then we want to approximate R = PQ^T
# Let's generate R.

seed(42)
ratings2 = []
for _ in range(100):
    user = []
    for _ in range(100):
        chance = gs(0, 0.4)
        
        # We'll fill our ratings matrix mostly with 0's to represent
        # unrated films. This is NOT standard; we're doing this only
        # to illustrate the general algorithm.
        if chance > 0.5:
            user.append(int(uni(1, 6)))
        else:
            user.append(0)
        
        # This 'if' will simply ensure that everyone has given at least
        # one rating.
        if user.count(0) == 10:
            user[int(uni(0, 10))] = int(uni(1, 6))
    ratings2.append(user)
ratings_arr2 = np.array(ratings2)

In [36]:
ratings_arr2

array([[0, 0, 0, ..., 0, 2, 0],
       [3, 0, 2, ..., 0, 0, 0],
       [5, 3, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 5, 0, 2],
       [0, 2, 5, ..., 0, 0, 0],
       [2, 2, 0, ..., 5, 0, 5]])

In [37]:
ratings_arr2.shape

(100, 100)

In [38]:
users2 = []

# Random generation of values for the user matrix
for _ in range(100):
    user = []
    for _ in range(10):
        user.append(gs(0, 1))
    users2.append(user)
users_arr2 = np.array(users2)

In [39]:
users_arr2

array([[ 8.82274940e-01, -6.95578968e-01,  1.54768279e+00,
         1.78060346e+00,  7.60860746e-01,  1.69049328e-03,
        -8.11676645e-01, -2.96592311e-01,  1.22018185e+00,
        -6.40643075e-01],
       [ 3.38046603e-01, -8.84372639e-02, -4.71171046e-01,
         2.97457087e-01,  7.47705044e-01, -1.52768298e+00,
         8.60478025e-01, -2.29952221e+00,  8.31657461e-02,
         1.75486978e+00],
       [-6.05034368e-01, -7.91079221e-01,  6.60531797e-01,
         1.77741409e+00,  2.32317539e+00,  1.10269697e+00,
        -3.46441549e-01,  2.08400708e+00,  1.17115157e+00,
         1.14886862e+00],
       [ 5.39182805e-01, -1.96643053e-01, -1.40011107e+00,
         5.20215810e-01,  9.77939247e-03, -1.46830679e-01,
         6.39371173e-01,  5.80257468e-01,  1.12484728e+00,
         8.45800019e-01],
       [-1.44557940e+00,  2.65330167e-01, -1.55962186e-01,
         4.92370669e-01, -4.90071582e-02, -1.09964415e+00,
         2.51199253e+00,  1.18982721e+00,  7.73182610e-01,
        -4.

In [40]:
pd.DataFrame(users_arr2)

,0,1,2,3,4,5,6,7,8,9
0,0.882275,-0.695579,1.547683,1.780603,0.760861,0.001690,-0.811677,-0.296592,1.220182,-0.640643
1,0.338047,-0.088437,-0.471171,0.297457,0.747705,-1.527683,0.860478,-2.299522,0.083166,1.754870
2,-0.605034,-0.791079,0.660532,1.777414,2.323175,1.102697,-0.346442,2.084007,1.171152,1.148869
3,0.539183,-0.196643,-1.400111,0.520216,0.009779,-0.146831,0.639371,0.580257,1.124847,0.845800
4,-1.445579,0.265330,-0.155962,0.492371,-0.049007,-1.099644,2.511993,1.189827,0.773183,-0.487234
...,...,...,...,...,...,...,...,...,...,...
95,-0.319777,1.574148,0.701796,-1.870593,-1.811309,1.939381,1.004103,0.420540,-0.594478,0.341561
96,-0.335925,0.647174,0.600024,-0.624072,2.314178,-1.354159,0.291670,-1.238657,-0.793236,-0.418211
97,0.178563,0.489958,-0.617309,1.273571,0.709245,-0.130058,-0.613932,-0.405129,0.490829,-1.048926
98,-0.194365,0.037390,-0.447188,-0.030845,0.001689,-1.397941,1.013686,0.568174,-1.682572,-1.264961


In [41]:
items2 = []

# Random generation of values for the item matrix
for _ in range(100):
    item = []
    for _ in range(10):
        item.append(gs(0, 1))
    items2.append(item)
items_arr2 = np.array(items2)

In [42]:
items_arr2

array([[-5.96830264e-01, -6.60644169e-01,  5.20090935e-01,
        -1.02332900e+00,  4.84720799e-01,  6.34063499e-01,
        -1.23738203e+00,  2.62804166e+00, -9.80718366e-01,
        -7.58465765e-01],
       [ 6.47152634e-01, -1.28732877e+00,  1.30196059e+00,
        -3.54192410e-01,  1.18282908e+00,  1.11582998e+00,
        -5.99358644e-02, -6.55814218e-01,  2.46163440e-01,
        -7.76689319e-01],
       [ 2.65328306e-01, -1.01459196e+00,  6.18856542e-02,
        -1.34603809e-01, -3.84773388e-02,  5.56442772e-01,
         1.43395593e+00,  2.29737204e-01,  3.14491196e-02,
        -4.03696762e-01],
       [-2.14052609e+00,  4.79318950e-01,  8.41153067e-01,
         1.12199449e+00, -1.26228248e+00, -4.56819541e-01,
         2.53746183e-02, -4.76825406e-01, -1.18486123e-01,
         7.94826506e-01],
       [-2.98006369e-01, -1.05783971e+00,  2.72952589e-01,
         4.47948214e-01,  4.94938380e-01, -1.40072089e-01,
         8.54957479e-01, -9.36771563e-01, -5.91063195e-01,
         6.

In [43]:
pd.DataFrame(items_arr2.T)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.596830,0.647153,0.265328,-2.140526,-0.298006,-0.232930,-1.883403,0.157618,-0.460766,-0.307029,...,-0.059417,-0.942780,-0.111691,-2.070029,-1.463647,-0.329576,-1.815151,-0.784101,0.229925,-0.866194
1,-0.660644,-1.287329,-1.014592,0.479319,-1.057840,-1.242306,-0.865101,1.668944,0.352837,-0.116424,...,0.550694,-0.438547,1.142652,-1.866966,-0.247959,2.348660,-0.653676,0.435620,-0.530768,-0.537740
2,0.520091,1.301961,0.061886,0.841153,0.272953,-0.994280,1.492607,0.894840,0.810954,0.393989,...,0.380713,0.263325,1.028478,1.474885,0.682138,-0.838659,0.484806,0.519732,-0.245760,0.480635
3,-1.023329,-0.354192,-0.134604,1.121994,0.447948,-1.044116,0.546491,-0.157221,-1.646296,-0.596573,...,-0.225170,-0.509486,0.453443,-2.014158,-1.147523,2.550073,2.453599,0.290546,0.041820,-0.666367
4,0.484721,1.182829,-0.038477,-1.262282,0.494938,-0.704526,0.245602,-1.422259,-0.469187,0.678312,...,-0.746622,-0.734250,2.259177,-0.064924,-0.744430,0.324012,-0.562406,-2.280384,0.463566,-0.461181
5,0.634063,1.115830,0.556443,-0.456820,-0.140072,0.542684,0.267765,0.869819,0.157358,0.185083,...,0.667552,-0.223644,-0.268150,1.277204,-1.651535,-0.017703,0.128135,0.271010,-0.191694,-0.384326
6,-1.237382,-0.059936,1.433956,0.025375,0.854957,0.136493,0.521970,-1.253742,0.660028,0.062560,...,0.820261,0.029104,-0.407943,1.407960,0.132683,-1.508440,1.017768,0.793832,0.184678,0.480799
7,2.628042,-0.655814,0.229737,-0.476825,-0.936772,-0.227285,0.237239,0.059475,-0.410627,0.572122,...,0.221031,-0.016949,0.016716,-1.177183,-0.738414,-0.979945,0.725833,0.351370,2.047333,0.749184
8,-0.980718,0.246163,0.031449,-0.118486,-0.591063,-0.265123,-2.348689,-1.415787,0.096965,0.809276,...,0.629898,1.809953,-2.074544,-0.717305,0.423223,-0.089482,-1.073721,0.821171,-2.517275,-0.297529
9,-0.758466,-0.776689,-0.403697,0.794827,0.638829,1.263870,-0.338494,-0.221315,1.435449,-0.856565,...,-0.189200,0.799876,0.538126,1.293203,-0.979161,1.862074,1.249311,-0.685380,0.128846,0.290700


Our first guess at filling in the matrix will simply be the matrix product:

In [44]:
guess = users_arr2.dot(items_arr2.T)

In [45]:
guess.shape == ratings_arr2.shape

True

In [46]:
guess

array([[-1.20022042,  4.79370844, -0.16745819, ..., -0.71515764,
        -3.29265957, -2.34612024],
       [-9.81954999, -1.09244517, -0.76883236, ..., -3.84048632,
        -3.63986143, -1.251285  ],
       [ 5.11924386,  2.8856345 ,  0.52294795, ..., -3.37782545,
         2.46092459, -0.03240239],
       ...,
       [-1.78409526,  0.16181232, -1.29355933, ..., -1.03771907,
        -1.97382977, -2.89026489],
       [ 1.85314949, -2.16815931,  1.15088825, ...,  0.0342376 ,
         5.73579374,  1.53629352],
       [-0.51939362, -3.31266136,  1.7857387 , ..., -0.18275941,
         2.55671716,  0.09216439]])

Let's get a measure of error:

In [47]:
ratings_arr2 - guess

array([[ 1.20022042, -4.79370844,  0.16745819, ...,  0.71515764,
         5.29265957,  2.34612024],
       [12.81954999,  1.09244517,  2.76883236, ...,  3.84048632,
         3.63986143,  1.251285  ],
       [-0.11924386,  0.1143655 , -0.52294795, ...,  3.37782545,
        -2.46092459,  0.03240239],
       ...,
       [ 1.78409526, -0.16181232,  1.29355933, ...,  6.03771907,
         1.97382977,  4.89026489],
       [-1.85314949,  4.16815931,  3.84911175, ..., -0.0342376 ,
        -5.73579374, -1.53629352],
       [ 2.51939362,  5.31266136, -1.7857387 , ...,  5.18275941,
        -2.55671716,  4.90783561]])

In [48]:
err = (ratings_arr2 - guess)**2

np.sum(err)

122333.98763617325

Pretty terrible! But we started with random numbers and have only done one iteration. Let's see if we can do better:

In [49]:
def als(ratings, users, items, reps=10):
    
    ratings_cols = ratings.T
    for _ in range(reps):
        new_users = []
        for i in range(len(ratings)):
            
            user = LinearRegression(fit_intercept=False)\
            .fit(items, ratings[i]).coef_
            new_users.append(user)
        new_users = np.asarray(new_users)
        
        new_items = []
        for i in range(len(ratings)):
            
            item = LinearRegression(fit_intercept=False)\
            .fit(new_users, ratings_cols[i]).coef_
            new_items.append(item)
        new_items = np.asarray(new_items)
        
        guess = new_users.dot(new_items.T)
        err = 0
        for i in range(len(ratings)):
            for j in range(len(ratings[i])):
                if ratings[i, j] != 0:
                    err += (ratings[i, j] - guess[i, j])**2
        print(err)
        
        items = new_items
        
    return new_users.dot(new_items.T)

We should see our error decrease with more iterations:

In [50]:
als(ratings_arr2, users_arr2, items_arr2, reps=100)

8679.959268046978
6511.270608291892
6272.528821101911
6190.959718973224
6153.148460896027
6133.158269096345
6121.902204676269
6115.079057339576
6110.53807458497
6107.25526650842
6104.734553278032
6102.714083690586
6101.043135121947
6099.629733317877
6098.4153980187575
6097.361480021129
6096.441351671074
6095.635877355333
6094.930768740715
6094.314999501616
6093.77979093405
6093.317895217616
6092.923040928008
6092.589488705116
6092.311688714318
6092.084047267304
6091.900807755377
6091.756039829785
6091.643717872174
6091.55786028533
6091.492697436179
6091.442838288918
6091.403412339846
6091.370172185086
6091.339550794579
6091.308674820064
6091.275340271621
6091.23795957393
6091.195489713938
6091.147350485181
6091.093340304618
6091.033555219434
6090.968314883924
6090.898097683235
6090.823485905555
6090.74512093579
6090.663667825518
6090.579788234867
6090.494120574916
6090.40726615071
6090.319780165164
6090.232166555164
6090.144875766741
6090.0583047180035
6089.97279833297
6089.88865215271

array([[ 0.21995462,  1.23529524, -0.40263864, ..., -0.07541482,
         0.07744066, -0.01191289],
       [ 2.87073575,  0.21173177,  1.76945471, ...,  0.97928197,
         0.20104584,  0.92905247],
       [ 6.77077847,  2.5889809 ,  0.18942039, ...,  0.05289979,
         0.83680627,  0.66474357],
       ...,
       [ 1.37968422, -0.33470618,  0.79340081, ...,  0.58204933,
         0.39782614,  0.4555666 ],
       [ 0.69625183,  2.60987891,  5.21806973, ...,  0.711433  ,
        -0.25176379,  0.03418347],
       [ 2.58584431,  1.06428306,  0.68004929, ...,  1.3844502 ,
        -0.4701946 ,  1.02692231]])